# Initial Exploration the Data Given

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
DATA_ROOT = "/Users/andrewgrowney/Data/kaggle/nfl-big-data-bowl-2024"

## Checkout the Out of Game Data

In [4]:
plays = pd.read_csv(f"{DATA_ROOT}/plays.csv")

In [5]:
plays.columns

Index(['gameId', 'playId', 'ballCarrierId', 'ballCarrierDisplayName',
       'playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam',
       'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'gameClock',
       'preSnapHomeScore', 'preSnapVisitorScore', 'passResult', 'passLength',
       'penaltyYards', 'prePenaltyPlayResult', 'playResult',
       'playNullifiedByPenalty', 'absoluteYardlineNumber', 'offenseFormation',
       'defendersInTheBox', 'passProbability', 'preSnapHomeTeamWinProbability',
       'preSnapVisitorTeamWinProbability', 'homeTeamWinProbabilityAdded',
       'visitorTeamWinProbilityAdded', 'expectedPoints', 'expectedPointsAdded',
       'foulName1', 'foulName2', 'foulNFLId1', 'foulNFLId2'],
      dtype='object')

In [6]:
plays[[
    "gameId", "gameClock", "down", "yardsToGo", "yardlineNumber",
    "possessionTeam", "playResult", "ballCarrierDisplayName",
    "passResult", "passLength", "passProbability",
    "offenseFormation", "defendersInTheBox",
    "expectedPoints", "expectedPointsAdded"]].head(5)

,gameId,gameClock,down,yardsToGo,yardlineNumber,possessionTeam,playResult,ballCarrierDisplayName,passResult,passLength,passProbability,offenseFormation,defendersInTheBox,expectedPoints,expectedPointsAdded
0,2022100908,7:52,1,10,41,ATL,9,Parker Hesse,C,6.0,0.747284,SHOTGUN,7.0,2.360609,0.981955
1,2022091103,7:38,1,10,34,PIT,3,Chase Claypool,NaN,NaN,0.416454,SHOTGUN,7.0,1.733344,-0.263424
2,2022091111,8:57,2,5,30,LV,15,Darren Waller,C,11.0,0.267933,I_FORM,6.0,1.312855,1.133666
3,2022100212,13:12,2,10,37,DEN,7,Mike Boone,NaN,NaN,0.592704,SINGLEBACK,6.0,1.641006,-0.043580
4,2022091900,8:33,1,10,35,BUF,3,Devin Singletary,NaN,NaN,0.470508,I_FORM,7.0,3.686428,-0.167903


# Look at week 1 tracking data to find out how to isolate motion plays

In [7]:
tw1 = pd.read_csv(f"{DATA_ROOT}/tracking_week_1.csv")

In [8]:
tw1["event"].unique()

array([nan, 'pass_arrived', 'pass_outcome_caught', 'tackle', 'run',
       'first_contact', 'ball_snap', 'handoff', 'touchdown',
       'out_of_bounds', 'man_in_motion', 'fumble', 'play_action',
       'pass_forward', 'lateral', 'autoevent_passforward',
       'autoevent_passinterrupted', 'line_set', 'qb_slide', 'shift',
       'run_pass_option', 'qb_sack', 'pass_shovel', 'autoevent_ballsnap',
       'snap_direct', 'fumble_defense_recovered',
       'fumble_offense_recovered'], dtype=object)

In [9]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import copy

# Color are courtesy of this kaggle post:
# https://www.kaggle.com/code/huntingdata11/animated-and-interactive-nfl-plays-in-plotly
colors = {
    'ARI':"#97233F", 
    'ATL':"#A71930", 
    'BAL':'#241773', 
    'BUF':"#00338D", 
    'CAR':"#0085CA", 
    'CHI':"#C83803", 
    'CIN':"#FB4F14", 
    'CLE':"#311D00", 
    'DAL':'#003594',
    'DEN':"#FB4F14", 
    'DET':"#0076B6", 
    'GB':"#203731", 
    'HOU':"#03202F", 
    'IND':"#002C5F", 
    'JAX':"#9F792C", 
    'KC':"#E31837", 
    'LA':"#003594", 
    'LAC':"#007FC8", 
    'LV':"#000000",
    'MIA':"#008E97", 
    'MIN':"#4F2683", 
    'NE':"#002244", 
    'NO':"#D3BC8D", 
    'NYG':"#0B2265", 
    'NYJ':"#125740", 
    'PHI':"#004C54", 
    'PIT':"#FFB612", 
    'SEA':"#69BE28", 
    'SF':"#AA0000",
    'TB':'#D50A0A', 
    'TEN':"#4B92DB", 
    'WAS':"#5A1414", 
    'football':'#CBB67C'
}

def get_blank_field():
    yardlines = np.arange(100, 1100+1, 100)
    yardline_width = 4
    
    yard_mark = list(np.arange(0, 50, 10)) + [50] + list(reversed(list(np.arange(0, 50, 10))))
    font_size=40

    # Draw a green rectangle
    field = Image.new("RGB", (1200, 533), "green")
    draw = ImageDraw.Draw(field)
    
    # Draw the yardlines and the yard marker text
    assert yardline_width % 2 == 0
    for yl, ym in zip(yardlines, yard_mark):
        yl_x = (yl - (yardline_width / 2))
        draw.line([(yl_x, 0), (yl_x, 533)], width = yardline_width, fill="white")
        
        font = ImageFont.truetype(f"{DATA_ROOT}/football_font.ttf", size=font_size)
        draw.text((yl-(font_size/2), 533-(font_size+5)), str(ym), font=font, fill = "black")
    
    # Flip the image so the text is right side up
    field = field.transpose(1)

    return field

def draw_play_frame(frame, highlight_ids = []):

    field = get_blank_field()
    draw = ImageDraw.Draw(field)

    p_rad = 6
    padding=2
    fb_w=8
    fb_h=5
    
    df = copy.deepcopy(frame)
    
    # Round the player locations to work in the image coordinates
    plot_x = df.loc[:, "x"].apply(lambda x: round(x, 1) * 10)
    df.loc[:, "plot_x"] = plot_x
    plot_y = df.loc[:, "y"].apply(lambda x: round(x, 1) * 10)
    df.loc[:, "plot_y"] = plot_y
        
    for row in df.iterrows():
        
        x = row[1]["plot_x"]
        y = row[1]["plot_y"]
        
        # Draw a white circle behind any player dots to be highlighted
        if row[1]["nflId"] in highlight_ids:
            draw.ellipse(((x-p_rad)-padding, (y-p_rad)-padding, (x+p_rad)+padding, (y+p_rad)+padding), fill="white")

        # Draw the football
        if row[1]["club"] == "football":
            draw.ellipse((x-fb_w, y-fb_h, x+fb_w, y+fb_h), fill=colors[row[1]["club"]])
        # Draw the players with color according to the colors dictionary
        else:
            draw.ellipse((x-p_rad, y-p_rad, x+p_rad, y+p_rad), fill=colors[row[1]["club"]])
        
    return field

def finalize(field, min_x = None, max_x = None):
    """
    Finalizes the image. Does the following
    - Flips the image along the x axis
    - Optionally crops out empty field according to min_x, max_x
    """
    field = field.transpose(1)
    if (min_x is not None) & (max_x is not None):
        field = field.crop((min_x, 0, max_x, 533))

    return field

def create_play_gif(play_player_tracking_df, gif_name, crop=False, highlight_ids=[]):
    """
    Draws the play frame by frame and saves to gif
    
    Parameters
    play_player_tracking_df - A df of player_tracking_data that contains 
    a unique gameId and a unique playId
    gif_name - The name of the gif, minus the .gif extension. This is 
    added automatically.
    crop - Whether or not to crop the gif to only contain the minimum and
    maximum x values within the entire play
    highlight_ids - The ids of players to draw a white circle behind them in
    order to call attention to them.
    
    """
    min_x = (round(play_player_tracking_df.x.min(), 1) * 10) - 50
    max_x = (round(play_player_tracking_df.x.max(), 1) * 10) + 50
    
    gif_frames = []
    frames = play_player_tracking_df["frameId"].values
    for i in frames:
        frame = play_player_tracking_df[play_player_tracking_df["frameId"] == i].copy(deep=True)
        field = get_blank_field()
        
        field = draw_play_frame(frame, highlight_ids)
            
        if crop:
            field = finalize(field, min_x = min_x, max_x =  max_x)
        else:
            field = finalize(field)
            
        gif_frames.append(field)
    frame_one = gif_frames[0]
    frame_one.save(f"{gif_name}.gif", format="GIF", append_images=gif_frames,
                save_all=True, duration=10, loop=0)

In [10]:
motion_plays = tw1.loc[tw1["event"] == "man_in_motion"][["gameId", "playId"]].drop_duplicates().values.tolist()

In [11]:

END_EVENTS = [
    'pass_arrived', 'pass_outcome_caught', 'tackle',
    'touchdown', 'out_of_bounds', 'fumble',
    'pass_forward', 'lateral', 'autoevent_passforward',
    'autoevent_passinterrupted', 'qb_slide', 'qb_sack',
    'pass_shovel', 'autoevent_ballsnap',
    'fumble_defense_recovered', 'fumble_offense_recovered']

def get_frame_id(play_df, event):
    event_row = play_df.loc[(play_df["event"] == event)]
    if event_row.shape[0] == 0:
        return None
    return event_row["frameId"].values[0]


# Find the frameId of the motion and snap
LIMIT = -1
for (game_id, play_id) in list(motion_plays)[:LIMIT]:
    full_play = tw1.loc[(tw1["playId"] == play_id) & (tw1["gameId"] == game_id)]
    motion_begin_frame          = get_frame_id(full_play, "man_in_motion")
    snap_frame                  = get_frame_id(full_play, "ball_snap")
    first_contact_frame         = get_frame_id(full_play, "first_contact")
    (play_end_frame, end_event) = full_play.loc[(full_play["frameId"] > snap_frame) & (full_play["event"].isin(END_EVENTS))].sort_values(by=["frameId"])[["frameId", "event"]].values[0]
    
    details = { 'Event Name': end_event, 'Motion Begin': motion_begin_frame, 'Snap': snap_frame, 'Play End': play_end_frame, 'First Contact': first_contact_frame}
    print(f"Game {game_id} Play {play_id}: {details}")


    motion_type = "presnap" if snap_frame > motion_begin_frame else "postsnap"
    start_frame = min(snap_frame, motion_begin_frame)
    motion = full_play.loc[(full_play["frameId"] >= start_frame) & (full_play["frameId"] <= play_end_frame)].sort_values("frameId")

    # Create the gif
    # create_play_gif(motion, f"gifs/{motion_type}/{game_id}_{play_id}", crop=True, highlight_ids=[])

Game 2022090800 Play 529: {'Event Name': 'tackle', 'Motion Begin': 1, 'Snap': 6, 'Play End': 50, 'First Contact': 37}
Game 2022090800 Play 2043: {'Event Name': 'tackle', 'Motion Begin': 8, 'Snap': 6, 'Play End': 48, 'First Contact': 43}
Game 2022090800 Play 2163: {'Event Name': 'tackle', 'Motion Begin': 3, 'Snap': 6, 'Play End': 68, 'First Contact': 40}
Game 2022090800 Play 2506: {'Event Name': 'tackle', 'Motion Begin': 4, 'Snap': 6, 'Play End': 56, 'First Contact': 37}
Game 2022090800 Play 2551: {'Event Name': 'tackle', 'Motion Begin': 4, 'Snap': 6, 'Play End': 59, 'First Contact': 31}
Game 2022091100 Play 501: {'Event Name': 'tackle', 'Motion Begin': 11, 'Snap': 6, 'Play End': 46, 'First Contact': 40}
Game 2022091100 Play 741: {'Event Name': 'tackle', 'Motion Begin': 5, 'Snap': 6, 'Play End': 55, 'First Contact': 38}
Game 2022091100 Play 2332: {'Event Name': 'tackle', 'Motion Begin': 10, 'Snap': 6, 'Play End': 68, 'First Contact': 48}
Game 2022091101 Play 1516: {'Event Name': 'tackle

In [12]:
print(len(motion_plays) * 8)

448


In [15]:
tw1.loc[tw1["event"] == "man_in_motion"][["gameId", "playId","displayName"]]

,gameId,playId,displayName
12834,2022090800,529,Rodger Saffold
12888,2022090800,529,Bobby Wagner
12942,2022090800,529,Aaron Donald
12996,2022090800,529,Mitch Morse
13050,2022090800,529,Troy Hill
13104,2022090800,529,Jake Kumerow
13158,2022090800,529,Jalen Ramsey
13212,2022090800,529,Leonard Floyd
13266,2022090800,529,A'Shawn Robinson
13320,2022090800,529,Dion Dawkins
